In [1]:
from class_definitions import *

In [8]:
Wines = [Wine(x) for x in Wine.pickled_wines]

In [9]:
len(Wines)

225

In [10]:
[w for w in Wines if w.wineSearcher == '403']

[Wine: Jacquère Cuvée Baujues 2016 St. Germain, Source: name-search,
 Wine: Bugey Cerdon Rosé Demi-Sec NV Dentelle, Source: name-search,
 Wine: Plavac Mali "Marica" 2017 Mrgudic, Source: name-search,
 Wine: Riesling Fass 2 Extra Trocken 2018 Lauer, Source: name-search,
 Wine: Muscadet Sèvre-et-Maine 2018 Batard Langelier, Source: name-search,
 Wine: C-d-P Tradition 750ml 2016 Bois de Boursan, Source: name-search,
 Wine: Silvaner Vulkan 2018 Helmut Dolde, Source: name-search,
 Wine: Malvasia "Bela" 2014 Rodica, Source: name-search,
 Wine: Naturalmente Frizzante Bianco 2018 Casa Belfi, Source: name-search,
 Wine: 2017 spatburgunder kabinett koehler ruprecht, Source: name-search,
 Wine: Sauvignon Blanc 2019 Le Petit Godon, Source: name-search,
 Wine: Sauvignon Blanc Pouch 1.5L Maivino, Source: name-search,
 Wine: Cabernet Sauvignon "Corrupto" 2019 Laurent, Source: name-search,
 Wine: Sincronia 2018 Mesquida Mora, Source: name-search,
 Wine: Vin de Savoie 2018 Jean Vullien, Source: name-se

In [11]:
kcw_wines = pd.read_csv('kings_county_wines.csv',index_col=0)

wine_names = list(kcw_wines['name'])

wineSearcher = pd.read_csv('wine-searcher_data.csv')

In [12]:
matched = [x for x in wine_names if x in wineSearcher['name'].to_list()]

In [13]:
# for wine_name in matched:
#     wine = Wine(wine_name)
#     if type(wine.wineSearcher) != dict:
#         print(wine.wineSearcher)
#         continue

In [14]:
wineSearcherData = pd.DataFrame.from_records([wine.wineSearcher for wine in Wines if type(wine.wineSearcher)==dict])
wineSearcherData.dropna(axis=0,thresh=3)

,url,name,Producer,Region/Appellation,Grape/Blend,Food Suggestion,Wine Style,Alcohol Content,Notes
1,NaN,Truffle Hound 2017 Unico Zelo,Unico Zelo,Clare Valley,Rare Red Blend,Beef and Venison,Red - Rich and Intense,12 - 13.5%,Indicative blend: Barbera and Nebbiolo
2,NaN,Sforzato di Valtellina 2013 Alfio Mozzi,Azienda Agricola Alfio Mozzi,Sforzato di Valtellina,Nebbiolo,Beef and Venison,Red - Rich and Intense,NaN,NaN
3,NaN,Cabernet Sauvignon 2017 Oracle Cellars,NaN,Mendocino,Cabernet Sauvignon,Beef and Venison,Red - Bold and Structured,NaN,NaN
5,NaN,Sauvignon Blanc 2019 Tablelands,TL Wine Co,New Zealand,Sauvignon Blanc,Goats' Cheese and Feta,White - Green and Flinty,12.5 - 14%,NaN
8,NaN,Moschofilero Fteri 2017 Troupis Winery,Troupis Winery,Peloponnese,Moschofilero,"Shellfish, Crab and Lobster",White - Green and Flinty,12 - 12.5%,NaN
...,...,...,...,...,...,...,...,...,...
193,NaN,"Caluso Passito ""Sule"" 2009 375ml Orsolani",Azienda Vitivinicola Orsolani,Erbaluce di Caluso,Erbaluce,Fruit-based Desserts,Dessert - Lush and Balanced,NaN,NaN
194,NaN,Sciaccarelu '1769' 2015 Clos Venturi,Domaine Vico,Vin de Corse,Sciaccarello (Mammolo),NaN,NaN,13.5%,NaN
195,NaN,"Soave Classico ""Otto"" 2018 Prá",Azienda Agricola Pra,Soave Classico,Soave Blend,White Fish,White - Green and Flinty,12%,Organic wine.
196,NaN,Tai Rosso 2018 Rezzadore,Azienda Agricola Rezzadore,Colli Berici,Grenache (Garnacha),Beef and Venison,Red - Rich and Intense,NaN,NaN


No result for wine-searcher, but one result for vivino. 
Re-instantiate from the vivino link for wine-searcher match?

In [15]:
wineSearcher_noResults = []
for W in Wines:
    if W.wineSearcher != '403':
        if len(W.wineSearcher.keys())==2:
            wineSearcher_noResults.append(W)

In [16]:
[(W.name,len(W.vivino)) for W in wineSearcher_noResults]

[('Zweigelt 2016 Bauer', 20),
 ('Spatburgunder Kabinett 2017 Koehler-Ruprecht', 1),
 ('Costa delle Sesia Uvaggio 2015 Sperino', 1),
 ('Aloxe Corton "Les Citernes" 2015 Bidault', 1),
 ('Refošk Baracija 2017 Clai Wines', 20),
 ('Lambrusco Bollarossa NV Corte Manzini', 1),
 ("Imereti 2018 Amiran's Wine Cellar", 20),
 ('Cinsault 2017 Turbigo', 1),
 ('Autour du Pigeonnier 2016 Benoît Roseau', 1),
 ('Garnacha "Campo de Borja" 2018 Dos Molinos', 1),
 ('Cab Sauv "Super Cali" Vogelzang 2013 CCGP', 20),
 ('Saint-Amour À La Folie 2017 Arpaye', 1),
 ('Piccolo Derthona 2017 Terralba', 1),
 ('Pet-Nat Orbis Moderandi 2019 OCD', 20),
 ('Fixin "Les Champstions" 2016 Domaine Collotte', 1),
 ("Catarratto L'Isola del Satiro 2018 Alcesti", 1),
 ("Sylvaner d'Alsace 2017 Vincent Fleith", 1),
 ('Riesling Kaefferkopf 1.5L 2016 Schoech', 2),
 ('Pinot Noir "La Belouse" 2017 L\'Echelette', 1),
 ('Langhe Nebbiolo 2017 Fratelli', 17),
 ('Rioja VS Tinto 2015 Vinos Subterraneos', 20),
 ('Jeanue Rouge 2017 Chateau Mus

In [17]:
easy_vivino_matches = []
for W in Wines:
    if len(W.vivino)==1:
        easy_vivino_matches.extend(W.vivino)

len(easy_vivino_matches)

91

### Scrape and store Vivino

In [133]:
import json 

In [212]:
def scrape_vivino(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    page = requests.get(url,headers=Wine.headers)
    bs = BeautifulSoup(page.content)
    
    ld_json = bs.find('script',attrs={'type':"application/ld+json"})
    overview = json.loads(ld_json.text)['mainEntity']
    
    scripts = bs.find_all('script')
    
    for s in scripts:
        if "winePageInformation" in s.text:
            info = s
    a = info.text.split(';')[1]
    b = a.strip('\n  window.__PRELOADED_STATE__.winePageInformation = ')
    
    wine_info = json.loads(b)
    
    return wine_info

In [214]:
result = scrape_vivino(158494162)

### selenium scraper 

In [285]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [502]:
chrome_driver_path = "/Users/schlinkertc/code/sm_selenium/chromedriver"
driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [503]:
driver.get(url)

In [504]:
master_wine_page = driver.find_element_by_id('master-wine-page-app')
driver.set_window_size(width=742,height=983)

a_tags = driver.find_elements_by_tag_name('a')

for a in a_tags:
    a_class = a.get_attribute('class')
    if 'toggleShowAll' in a_class:
        show_all = a
        break

a.click()


In [506]:
text = master_wine_page.text

In [508]:
[x for x in text.split('\n') if 'mentions of' in x]

['8 mentions of black fruit notes',
 '6 mentions of red fruit notes',
 '6 mentions of oaky notes',
 '6 mentions of earthy notes',
 '5 mentions of spices notes',
 '2 mentions of microbio notes',
 '1 mentions of floral notes',
 '1 mentions of vegetal notes',
 '1 mentions of dried fruit notes']

In [494]:
driver.set_window_size(width=742,height=983)

svgs = master_wine_page.find_elements_by_tag_name('svg')
[s.get_attribute('class') for s in svgs]

tasteNotes = []
for s in svgs:
    try:
        svg_class = s.get_attribute('class')
        if 'tasteNote' in svg_class:
        
            tasteNotes.append(s)
    except:
        continue 

In [475]:
import time

In [476]:
time.sleep(5)

In [479]:
a_tags = driver.find_elements_by_tag_name('a')

for a in a_tags:
    a_class = a.get_attribute('class')
    if 'toggleShowAll' in a_class:
        show_all = a
        break

a.click()

note_texts = []
for i in range(len(tasteNotes)):
    tasteNotes[i].click()
    divs = driver.find_elements_by_tag_name('div')
    note_text = divs[1].text
    note_texts.append(note_text)
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    time.sleep(3)

In [486]:
len(note_texts)

9

In [487]:
spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]

styles = []
for span in spans:
    try:
        style = span.get_attribute('style')
        if style.startswith('width'):
            styles.append(style)
    except:
        continue

styles

In [491]:
driver.quit()

In [500]:
def selenium_scrape(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    
    chrome_driver_path = "/Users/schlinkertc/code/sm_selenium/chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    
    driver.get(url)
    
    master_wine_page = driver.find_element_by_id('master-wine-page-app')
    
    driver.set_window_size(width=742,height=983)

    svgs = master_wine_page.find_elements_by_tag_name('svg')
    [s.get_attribute('class') for s in svgs]

    tasteNotes = []
    for s in svgs:
        try:
            svg_class = s.get_attribute('class')
            if 'tasteNote' in svg_class:
                tasteNotes.append(s)
        except:
            continue 
            
    a_tags = driver.find_elements_by_tag_name('a')

    for a in a_tags:
        a_class = a.get_attribute('class')
        if 'toggleShowAll' in a_class:
            show_all = a
            break

    a.click()

    note_texts = []
    for i in range(len(tasteNotes)):
        tasteNotes[i].click()
        divs = driver.find_elements_by_tag_name('div')
        note_text = divs[1].text
        note_texts.append(note_text.split('\n'))
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        time.sleep(3)
        
    spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]

    styles = []
    for span in spans:
        try:
            style = span.get_attribute('style')
            if style.startswith('width'):
                styles.append(style)
        except:
            continue
    driver.quit()
        
    return note_texts,styles

In [501]:
selenium_scrape(158494162)

([['cart icon',
   'blackberries_72pt',
   'Medium ruby color. Blackberry, violet, graphite, and deep earthy - almost cave-like earthy - black cherry nose. Medium + tannins. Medium high acidity. Medium + body. Blackberry, bramble, rich leather, flint, and black cherry. Blue fruits and earthy minerals. 3.7⭐️ great flavors but something about the structure is throwing me off. I do love the 20% Mourvèdre in this wine. 60% Grenache, 20% Syrah, 20% Mourvèdre. 14% abv',
   '29',
   '1',
   'Ed S',
   '23 Sep 20192017 vintage',
   'plum, blackberry, currant, earth.',
   '0',
   '0',
   'Tudor Popescu',
   '1 Feb 20202017 vintage',
   'M- body. Tart bright black cherry.',
   '0',
   '0',
   'Kevin Smith',
   '28 Feb 20202017 vintage',
   'Black pepper, cherry, plum, oak.',
   '0',
   '0',
   'Stuart Meikle',
   '16 May 20192017 vintage',
   'Black Tea prunes and dark fruit balance nicely.',
   '0',
   '0',
   'Allison Mickel',
   '15 Feb 20202017 vintage',
   'Jammy, plum, well balanced',
   '